In [192]:
import numpy as np
import pickle
import pandas as pd

# input file 1: DEG matrix in csv format
df = pd.read_csv("deg_324.csv", index_col=0)

tmr = list(df.index)

idx2deg = {i:d for i, d in enumerate(df.columns)}

deg = df.values

In [193]:
# input file 2: cancer type list in txt format
can = []
with open("cancer_type_324.txt", "r") as f:
    for line in f:
        can.append(line.strip())

idx2can = {i:c for i, c in enumerate(list(set(can)))}
can2idx = {idx2can[i]:i for i in idx2can.keys()}

can = [can2idx[c] for c in can]

can = np.array(can)

In [194]:
# input file 3: SGA list in txt format
sga = []
with open("sga_324_new.txt", "r") as f:
    # next(f)
    for line in f:
        line = line.strip().split(", ")
        sga.append(line[:])

idx2sga = {i:s for i,s in enumerate(list(set([l for line in sga for l in line])))}
sga2idx = {idx2sga[i]:i for i in idx2sga.keys()}

sga = [[sga2idx[l] for l in line] for line in sga]

In [195]:
# output file: dictionary in pickle format
dataset = {
    "can": can,
    "deg": deg,
    "idx2can": idx2can,
    "idx2deg": idx2deg,
    "idx2sga": idx2sga,
    "sga": sga,
    "tmr": tmr}

with open("dataset_324_new.pkl", "wb") as f:
    pickle.dump(dataset, f, protocol=2)

In [196]:
gene_emb_pt_1_epoch = np.load('gene_emb_pretrain_1_epoch.npy')

In [197]:
with open('idx2gene.txt') as f:
    idx2gene = f.readlines()[0]

In [198]:
idx = list(map(lambda x: int(x.split(': ')[0]), idx2gene.split(', ')))
genes = list(map(lambda x: x.split(': ')[1][1:-1], idx2gene.split(', ')))

In [199]:
idx2gene = {}
gene2idx = {}
for i in range(len(idx)):
    gene = genes[i]
    idx2gene[idx[i]] = gene
    gene2idx[gene] = idx[i]

In [200]:
gene_embs = [[0]*512]
not_found = []
for i in range(len(idx2sga.keys())):
    gene = idx2sga[i]
    try:
        new_idx = gene2idx[gene]
    except:
        not_found.append(gene)
    gene_embs.append(gene_emb_pt_1_epoch[new_idx])

In [201]:
gene_emb_pretrain = np.array(gene_embs)
gene_emb_pretrain.shape

(324, 512)

In [202]:
with open('gene_emb_pretrain_324_new.npy', 'wb') as f:
    np.save(f, gene_emb_pretrain)

In [105]:
### adjust the SGAs to the 324 genes 

# input file 1: DEG matrix in csv format [no change]
df = pd.read_csv("deg.csv", index_col=0)

tmr = list(df.index)

idx2deg = {i:d for i, d in enumerate(df.columns)}

deg = df.values

In [102]:
# input file 2: cancer type list in txt format [no change]
can = []
with open("cancer_type.txt", "r") as f: 
    for line in f:
        can.append(line.strip())

idx2can = {i:c for i, c in enumerate(list(set(can)))}
can2idx = {idx2can[i]:i for i in idx2can.keys()}

can = [can2idx[c] for c in can]

can = np.array(can)

In [92]:
mutation_df = pd.read_csv('../../data sets/mutations_raw_324_final.csv')

In [93]:
mutation_df.head()

,DepMap_ID,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,APC,...,VEGFA,VHL,WHSC1,WHSC1L1,WT1,XPO1,XRCC2,ZNF217,ZNF703,TERC
0,ACH-000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,ACH-000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,ACH-000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,ACH-000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,ACH-000007,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [95]:
genes_324 = mutation_df.columns[1:]

In [126]:
# input file 3: SGA list in txt format
name_dict = {'FAM123B': 'AMER1', 'FAM46C': 'TENT5C', 'C17orf39': 'GID4', 'H3F3A': 'H3-3A', 'MLL': 'KMT2A',
            'MLL2': 'KMT2D', 'MRE11A': 'MRE11', 'MYCL1': 'MYCL', 'PARK2': 'PRKN', 'WHSC1': 'NSD2', 'WHSC1L1': 'NSD3'}
track_dict = {}

sga = []
with open("sga.txt", "r") as f:
    next(f)
    for line in f:
        line = line.strip().split(", ")
        new_line = line[1:]
        add_line = []
        for gene in new_line:
            if gene in genes_324:
                add_line.append(gene)
            elif gene in name_dict.values():
                add_line.append(gene)
                track_dict[gene] = 1
        sga.append(add_line)

idx2sga = {i:s for i,s in enumerate(list(set([l for line in sga for l in line])))}
sga2idx = {idx2sga[i]:i for i in idx2sga.keys()}

sga = [[sga2idx[l] for l in line] for line in sga]

In [113]:
# output file: dictionary in pickle format
dataset = {
    "can": can,
    "deg": deg,
    "idx2can": idx2can,
    "idx2deg": idx2deg,
    "idx2sga": idx2sga,
    "sga": sga,
    "tmr": tmr}

with open("dataset_adjusted.pkl", "wb") as f:
    pickle.dump(dataset, f, protocol=2)

In [114]:
gene_emb_pretrain_ori = np.load('gene_emb_pretrain.npy')

In [115]:
with open('idx2gene.txt') as f:
    idx2gene = f.readlines()[0]

In [116]:
idx = list(map(lambda x: int(x.split(': ')[0]), idx2gene.split(', ')))
genes = list(map(lambda x: x.split(': ')[1][1:-1], idx2gene.split(', ')))

In [117]:
idx2gene = {}
gene2idx = {}
for i in range(len(idx)):
    gene = genes[i]
    idx2gene[idx[i]] = gene
    gene2idx[gene] = idx[i]

In [118]:
gene_embs = [[0]*512]
not_found = []
for i in range(len(idx2sga.keys())):
    gene = idx2sga[i]
    try:
        new_idx = gene2idx[gene]
    except:
        not_found.append(gene)
    gene_embs.append(gene_emb_pretrain_ori[new_idx])

In [119]:
gene_emb_pretrain = np.array(gene_embs)
gene_emb_pretrain.shape

(306, 512)

In [124]:
with open('gene_emb_pretrain_adjusted.npy', 'wb') as f:
    np.save(f, gene_emb_pretrain)

In [137]:
### adjust the SGAs to the 324 genes 
# input file 1: DEG matrix in csv format [no change]
df = pd.read_csv("deg.csv", index_col=0)

tmr = list(df.index)

idx2deg = {i:d for i, d in enumerate(df.columns)}

deg = df.values

In [138]:
deg.shape

(4468, 2207)

In [139]:
# input file 2: cancer type list in txt format [no change]
can = []
with open("cancer_type.txt", "r") as f: 
    for line in f:
        can.append(line.strip())

idx2can = {i:c for i, c in enumerate(list(set(can)))}
can2idx = {idx2can[i]:i for i in idx2can.keys()}

can = [can2idx[c] for c in can]

can = np.array(can)

In [140]:
can.shape

(4468,)

In [161]:
# input file 3: SGA list in txt format (ori)
sga_ori = []
with open("sga.txt", "r") as f:
    next(f)
    for line in f:
        line = line.strip().split(", ")
        sga_ori.append(line[1:])

idx2sga_ori = {i:s for i,s in enumerate(list(set([l for line in sga_ori for l in line])))}
sga2idx_ori = {idx2sga_ori[i]:i for i in idx2sga_ori.keys()}

sga_ori = [[sga2idx_ori[l] for l in line] for line in sga_ori]

In [162]:
len(sga_ori)

4468

In [163]:
max_id = 0
for i in sga_ori:
    max_id = max(max_id, max(i))
max_id

19780

In [185]:
# input file 3: SGA list in txt format
missing_genes = list(genes_324)
name_dict = {'FAM123B': 'AMER1', 'FAM46C': 'TENT5C', 'C17orf39': 'GID4', 'H3F3A': 'H3-3A', 'MLL': 'KMT2A',
            'MLL2': 'KMT2D', 'MRE11A': 'MRE11', 'MYCL1': 'MYCL', 'PARK2': 'PRKN', 'WHSC1': 'NSD2', 'WHSC1L1': 'NSD3'}

sga = []
with open("sga.txt", "r") as f:
    next(f)
    for line in f:
        line = line.strip().split(", ")
        new_line = line[1:]
        add_line = []
        for gene in new_line:
            if gene in genes_324:
                add_line.append(gene)
                if gene in missing_genes:
                    missing_genes.remove(gene)
            elif gene in name_dict.values():
                add_line.append(gene)
                if gene in missing_genes:
                    missing_genes.remove(gene)
        sga.append(add_line)

idx2sga = {i:s for i,s in enumerate(list(set([l for line in sga for l in line])))}
sga2idx = {idx2sga[i]:i for i in idx2sga.keys()}

sga = [[sga2idx[l] for l in line] for line in sga]

In [186]:
print(missing_genes)

['BCL2L2', 'EMSY', 'CCND1', 'CD274', 'CD70', 'FGF19', 'FGF3', 'FGF4', 'GATA4', 'ID3', 'MCL1', 'MDM4', 'MYC', 'PDCD1LG2', 'RAD21', 'RSPO2', 'SDHC', 'SDHD', 'SOCS1', 'SOX2', 'TNFRSF14', 'ZNF703', 'TERC']


In [187]:
max_id = 0
for i in sga:
    if i:
        max_id = max(max_id, max(i))
max_id

304